In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict

In [8]:
categorical_columns = ['map', 'time_left']

Transform the time_left column from continius to 6 intervals

In [3]:
data_train = pd.read_csv("modeling_set.csv")

In [4]:
data_train.time_left = pd.cut(data_train.time_left,bins=[0,50,75,100,125, 150, 175],labels=[1,2,3,4,5,6])

In [9]:
columns_to_add = defaultdict(list)
for col in categorical_columns:
    dummies = pd.get_dummies(data_train[col], prefix=col)
    data_train = pd.concat([data_train, dummies], axis=1)
    data_train.drop(col, axis=1, inplace=True)
    columns_to_add[col] += [new_col.replace(col + "_", "") for new_col in dummies.columns]

In [5]:
data_train.round_winner = data_train.round_winner.astype('category').cat.codes

In [11]:
data_test = pd.read_csv("validation_set.csv")

In [12]:
data_test.time_left = pd.cut(data_test.time_left,bins=[0,50,75,100,125, 150, 175],labels=[1,2,3,4,5,6])

In [13]:
columns_to_add = defaultdict(list)
for col in categorical_columns:
    dummies = pd.get_dummies(data_test[col], prefix=col)
    data_test = pd.concat([data_test, dummies], axis=1)
    data_test.drop(col, axis=1, inplace=True)
    columns_to_add[col] += [new_col.replace(col + "_", "") for new_col in dummies.columns]

In [29]:
data_test.round_winner = data_test.round_winner.astype('category').cat.codes

In [23]:
X_train = train_set.drop("round_winner", axis=1)
X_test = test_set.drop("round_winner", axis=1)
y_train = train_set["round_winner"]
y_test = test_set["round_winner"]

Random Forrest

In [24]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 100, max_features = 'auto', max_depth = 20, criterion = 'entropy', min_samples_split = 3, random_state=0)

In [25]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [100, 200, 300],
    'max_features': ['auto'],
    'max_depth' : [10, 20, 30],
    'criterion' :['entropy'], 
    "min_samples_split": [2, 3, 5, 10]
}


In [27]:
from sklearn.model_selection import GridSearchCV
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, y_train)
CV_rfc.best_params_

{'criterion': 'entropy',
 'max_depth': 30,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 300}

In [33]:
from sklearn.metrics import accuracy_score
y_pred = rfc.fit(X_train, y_train).predict(X_test)
print("#Mislabeled points (RF) : {1} out of {0}".format(len(y_test), sum(y_pred != y_test)))
print("Model Accuray: {:.4f}".format(accuracy_score(y_test, y_pred)))

#Mislabeled points (RF) : 4782 out of 36723
Model Accuray: 0.8698


In [37]:
import pickle
filename = 'storage/rfc.sav'
pickle.dump(rfc, open(filename, 'wb'))